In [23]:
import pickle
import pandas as pd
import gc
from sklearn.metrics import classification_report

In [24]:
X = pd.read_pickle('/kaggle/input/amex-imputed-aggregate-data/X_train_agg.pkl', compression='gzip')
y = pd.read_pickle('/kaggle/input/amex-imputed-aggregate-data/y_train_agg.pkl', compression='gzip')

Function to calculate competition's evaluation metric: https://www.kaggle.com/code/inversion/amex-competition-metric-python

In [25]:
def amex_metric(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:

    def top_four_percent_captured(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
        
    def weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        df = (pd.concat([y_true, y_pred], axis='columns')
              .sort_values('prediction', ascending=False))
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        df['random'] = (df['weight'] / df['weight'].sum()).cumsum()
        total_pos = (df['target'] * df['weight']).sum()
        df['cum_pos_found'] = (df['target'] * df['weight']).cumsum()
        df['lorentz'] = df['cum_pos_found'] / total_pos
        df['gini'] = (df['lorentz'] - df['random']) * df['weight']
        return df['gini'].sum()

    def normalized_weighted_gini(y_true: pd.DataFrame, y_pred: pd.DataFrame) -> float:
        y_true_pred = y_true.rename(columns={'target': 'prediction'})
        return weighted_gini(y_true, y_pred) / weighted_gini(y_true, y_true_pred)

    g = normalized_weighted_gini(y_true, y_pred)
    d = top_four_percent_captured(y_true, y_pred)

    return 0.5 * (g + d)

In [26]:
# build the lightgbm model
import lightgbm as lgb
model = lgb.LGBMClassifier()
model.fit(X, y)

LGBMClassifier()

In [27]:
preds = model.predict(X)
print(f'M = {amex_metric(pd.DataFrame(y), pd.DataFrame(preds, index=y.index, columns=["prediction"]))}')

M = 0.588492095944125


In [28]:
print(classification_report(y, preds))

              precision    recall  f1-score   support

           0       0.94      0.94      0.94    340085
           1       0.82      0.82      0.82    118828

    accuracy                           0.91    458913
   macro avg       0.88      0.88      0.88    458913
weighted avg       0.91      0.91      0.91    458913



In [29]:
del X, y
gc.collect()

113

Save model and predict on test set¶

In [30]:
pickle.dump(model, open('LGBM_agg_data.sav', 'wb'))

In [31]:
X_test = pd.read_pickle('/kaggle/input/amex-imputed-aggregate-data/X_test_agg.pkl', compression='gzip')

In [37]:
submission = pd.DataFrame(model.predict_proba(X_test)[:,1], index=X_test.index, columns=['prediction'])

# index needs to be removed from submission csv
submission = submission.reset_index()
submission.to_csv('baseline_LGBM_Classifer_Agg.csv', index=False)